In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from collections import namedtuple
import numpy as np
import os
from sklearn.utils import shuffle

Dataset = namedtuple("Dataset", ["x", "r", "fi"])

In [ ]:
sweep_config = {

    'method': 'bayes',
    'metric': {
      'name': 'val_mae',
      'goal': 'minimize'   
    },
    'parameters': {
        'target': {
            'values': ['fi']
        },
        'epochs': {
           'values': [100]
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'layer_1_size': {
                'values': [5040, 5760, 6480]
        },
        'layer_2_size': {
                'values': [5760, 6480, 7200]
        },
        'layer_3_size': {
                'values': [1440, 2160, 2880, 3600]
        },
        'learning_rate': {
            'distribution': 'uniform',
            'max': 0.002,
            'min': 2e-06
        },
        'optimizer': {
            'distribution': 'categorical',
            'values': ['sgd', 'rmsprop']
        },
        'activation1': {
            'distribution': 'categorical',
            'values': ['tanh', 'relu']
        },
        'activation2': {
            'distribution': 'categorical',
            'values': ['relu', 'tanh']
        },
        'activation3': {
            'distribution': 'categorical',
            'values': ['relu']
        },
        'activation4': {
            'distribution': 'categorical',
            'values': ['relu']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="artem-starkov", project="flatfasetgen")

In [ ]:
def read(data_dir, split):
    filename = split + ".npz"
    data = np.load(os.path.join(data_dir, filename))

    return Dataset(x=data["x"], r=data["r"], fi=data["fi"])

In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 50,
        'batch_size': 16,
        'learning_rate': 1e-6,
        'activation1': 'relu',
        'activation2': 'relu',
        'activation3': 'relu',
        'activation4': 'relu',
        'optimizer': 'sgd',
        'layer_1_size': 4096,
        'layer_2_size': 4096,
        'layer_3_size': 4096,
        'm': 720,
        'h': 40,
        'l' : 20,
        'rMin': 50,
        'rMax': 1000,
        'gMax': 100,
        'target': 'fi'
    }

    run = wandb.init(project="flatfasetgen", job_type="training_angle_w/o_d.out", config=config_defaults)
    processed_data = wandb.Artifact("Clear_datasets", type="dataset")
    raw_data_artifact = run.use_artifact('Clear_datasets:v20') 
    raw_dataset = raw_data_artifact.download()
    train_dataset = read(raw_dataset, 'train_dataset')
    test_dataset = read(raw_dataset, 'test_dataset')
    run.log_artifact(processed_data)
    X_train, X_test, y_train, y_test = train_dataset.x, test_dataset.x, train_dataset.r, test_dataset.r
    config = wandb.config
    model = Sequential()
    model.add(Dense(config.m, input_shape=(config.m,)))
    model.add(Activation(config.activation1))

    model.add(Dense(config.layer_1_size))
    model.add(Activation(config.activation2))

    model.add(Dense(config.layer_2_size))
    model.add(Activation(config.activation3))

    model.add(Dense(config.layer_3_size))
    model.add(Activation(config.activation4))


    model.add(Dense(1))

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = SGD(lr=config.learning_rate, decay=1e-5, nesterov=True)
    elif config.optimizer=='rmsprop':
      optimizer = RMSprop(lr=config.learning_rate, decay=1e-5)

    model.compile(loss='mae', optimizer = optimizer, metrics=['mae', 'mape'])

    model_artifact = wandb.Artifact(
            "angle_compiled_model", type="model",
            description=f"bayes, 100k, best params from skratch, target: {config.target}",
            metadata=dict(config))
    model.save("angle_compiled_models")
    model_artifact.add_dir("angle_compiled_models")
    run.log_artifact(model_artifact)

    model.fit(X_train, y_train, batch_size=config.batch_size,
              epochs=config.epochs, validation_data=(X_test, y_test),
              callbacks=[WandbCallback(validation_data=(X_test, y_test)),
                          EarlyStopping(patience=10, restore_best_weights=True)])
    wandb.log({"R2" : r2_score(y_test, model.predict(X_test))})
    wandb.log({"RMSE": mean_squared_error(y_test, model.predict(X_test), squared=False)})
    trained_model_artifact = wandb.Artifact(
            "angle_trained_model", type="model",
            description="",
            metadata=dict(config))
    
    model.save('angle_trained_models')
    trained_model_artifact.add_dir('angle_trained_models')
    run.log_artifact(trained_model_artifact)
    run.finish()

In [ ]:
wandb.agent(sweep_id, train, count=10)